# NATMI 

In [1]:
lib = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/'
import sys
sys.path.append(lib)
from utilities.package_importing import *
import scipy.stats as stats

### All pairs:

In [2]:
ROOT_PATH = r'/storage/md_keren/shitay/outputs/NATMI/tumor_CD8_myeloid/executions/21.3.22'
FILE = 'Edges_lrc2p.csv'
df = pd.read_csv(join(ROOT_PATH, FILE))

In [3]:
df#.sort_values('Receptor detection rate', ascending = False).iloc[:, 4:]

Sending cluster Ligand symbol Receptor symbol     Target cluster  \
0           CD8_cluster_1           A2M            LRP1      CD8_cluster_1   
1           CD8_cluster_1           A2M            LRP1      CD8_cluster_2   
2           CD8_cluster_1           A2M            LRP1      CD8_cluster_3   
3           CD8_cluster_1           A2M            LRP1      CD8_cluster_4   
4           CD8_cluster_1           A2M            LRP1      CD8_cluster_5   
...                   ...           ...             ...                ...   
995531  Myeloid_cluster_9           ZP3           MERTK  Myeloid_cluster_5   
995532  Myeloid_cluster_9           ZP3           MERTK  Myeloid_cluster_6   
995533  Myeloid_cluster_9           ZP3           MERTK  Myeloid_cluster_7   
995534  Myeloid_cluster_9           ZP3           MERTK  Myeloid_cluster_8   
995535  Myeloid_cluster_9           ZP3           MERTK  Myeloid_cluster_9   

        Ligand-expressing cells  Ligand detection rate  \
0                            16               0.004857   
1                            16               0.004857   
2                            16               0.004857   
3                            16               0.004857   
4                            16               0.004857   
...                         ...                    ...   
995531                       44               0.035743   
995532                       44               0.035743   
995533                       44               0.035743   
995534                       44               0.035743   
995535                       44               0.035743   

        Ligand average expression value  Ligand total expression value  \
0                              0.012162                      40.062253   
1                              0.012162                      40.062253   
2                              0.012162                      40.062253   
3                              0.012162                      40.062253   
4                              0.012162                      40.062253   
...                                 ...                            ...   
995531                         0.024053                      29.609107   
995532                         0.024053                      29.609107   
995533                         0.024053                      29.609107   
995534                         0.024053                      29.609107   
995535                         0.024053                      29.609107   

        Ligand derived specificity of average expression value  \
0                                                0.000576        
1                                                0.000576        
2                                                0.000576        
3                                                0.000576        
4                                                0.000576        
...                                                   ...        
995531                                           0.028669        
995532                                           0.028669        
995533                                           0.028669        
995534                                           0.028669        
995535                                           0.028669        

        Ligand derived specificity of total expression value  \
0                                                0.000810      
1                                                0.000810      
2                                                0.000810      
3                                                0.000810      
4                                                0.000810      
...                                                   ...      
995531                                           0.014232      
995532                                           0.014232      
995533                                           0.014232      
995534                                           0.014

##### Distributions:

In [4]:
sns.reset_orig() # reset size of figure
columns = ['Ligand detection rate', 'Receptor detection rate',
           'Ligand average expression value', 'Receptor average expression value', 
           'Ligand total expression value','Receptor total expression value',
           'Ligand derived specificity of average expression value','Receptor derived specificity of average expression value',
           'Ligand derived specificity of total expression value', 'Receptor derived specificity of total expression value',          
           
           'Edge average expression weight',
           'Edge average expression derived specificity',
           'Edge total expression weight',
           'Edge total expression derived specificity']

fig, ax = plt.subplots(int(len(columns)/2), 2)
fig.set_size_inches(12, 26)
ax = ax.ravel()

for idx in range(len(columns)):
    sns.boxplot(ax=ax[idx], y=columns[idx], data=df).set(title=columns[idx], ylabel='');

##### ROI:

In [5]:
df.columns

Index(['Sending cluster', 'Ligand symbol', 'Receptor symbol', 'Target cluster',
       'Ligand-expressing cells', 'Ligand detection rate',
       'Ligand average expression value', 'Ligand total expression value',
       'Ligand derived specificity of average expression value',
       'Ligand derived specificity of total expression value',
       'Receptor-expressing cells', 'Receptor detection rate',
       'Receptor average expression value', 'Receptor total expression value',
       'Receptor derived specificity of average expression value',
       'Receptor derived specificity of total expression value',
       'Edge average expression weight',
       'Edge average expression derived specificity',
       'Edge total expression weight',
       'Edge total expression derived specificity'],
      dtype='object')

In [6]:
sort_by = 'Edge average expression weight'
# sort_by = 'Edge average expression derived specificity'
# sort_by = 'Edge total expression weight'
# sort_by = 'Edge total expression derived specificity'

avg_specificity_threshold  = 0 #0.01
total_specificity_threshold = 0 #0.02
detection_threshold = 0.7
avg_expression_threshold = 1

filter_GEP = False


################################ filter ################################

df_filtered = df[(df['Receptor detection rate']>detection_threshold) & 
                 (df['Ligand detection rate']>detection_threshold)]

df_filtered = df_filtered[(df_filtered['Receptor average expression value']>avg_expression_threshold) &
                          (df_filtered['Ligand average expression value']>avg_expression_threshold)]
# df_filtered = df_filtered[(df_filtered['Receptor total expression value']>avg_expression_threshold) &
#                           (df_filtered['Ligand total expression value']>avg_expression_threshold)]

df_filtered = df_filtered[(df_filtered['Receptor derived specificity of average expression value']>avg_specificity_threshold) &
                          (df_filtered['Ligand derived specificity of average expression value']>avg_specificity_threshold)]

df_filtered = df_filtered[(df_filtered['Receptor derived specificity of total expression value']>total_specificity_threshold) &
                          (df_filtered['Ligand derived specificity of total expression value']>total_specificity_threshold)]


if filter_GEP:
    print(f'GEP siganls are filtered out')
    df_filtered = df_filtered[df_filtered[['Sending cluster', 'Target cluster']].apply(lambda x: not 'GEP' in x['Sending cluster'] and not 'GEP' in x['Target cluster'], axis=1)]

print(f'Number of pairs in filtered df is: {len(df_filtered)}')
################################ show ################################



pd.set_option('display.max_rows', 10)
strong_signals_df = df_filtered.sort_values(sort_by, ascending = False).reset_index(drop=True)
strong_signals_df

Number of pairs in filtered df is: 3753


Sending cluster Ligand symbol Receptor symbol     Target cluster  \
0            GEP_#18           MIF            CD74  Myeloid_cluster_1   
1            GEP_#13           MIF            CD74  Myeloid_cluster_1   
2            GEP_#18           MIF            CD74  Myeloid_cluster_7   
3            GEP_#12           MIF            CD74  Myeloid_cluster_1   
4            GEP_#13           MIF            CD74  Myeloid_cluster_7   
...              ...           ...             ...                ...   
3748         GEP_#11          DSC3            DSG2            GEP_#11   
3749          GEP_#4           GRN        TNFRSF1A  Myeloid_cluster_9   
3750         GEP_#11           GRN        TNFRSF1A  Myeloid_cluster_9   
3751         GEP_#11         ADAM9           ITGA3            GEP_#11   
3752         GEP_#11         CXADR           CXADR            GEP_#11   

      Ligand-expressing cells  Ligand detection rate  \
0                        5643               0.997349   
1                        2946               0.990918   
2                        5643               0.997349   
3                        7434               0.995447   
4                        2946               0.990918   
...                       ...                    ...   
3748                       78               0.829787   
3749                     2363               0.764725   
3750                       77               0.819149   
3751                       75               0.797872   
3752                       77               0.819149   

      Ligand average expression value  Ligand total expression value  \
0                            5.785267                   32733.040182   
1                            5.668784                   16853.295504   
2                            5.785267                   32733.040182   
3                            5.508015                   41133.854818   
4                            5.668784                   16853.295504   
...                               ...                            ...   
3748                         1.018530                      95.741827   
3749                         1.089482                    3366.499399   
3750                         1.079144                     101.439503   
3751                         1.073000                     100.862024   
3752                         1.036579                      97.438429   

      Ligand derived specificity of average expression value  \
0                                              0.046097        
1                                              0.045169        
2                                              0.046097        
3                                              0.043888        
4                                              0.045169        
...                                                 ...        
3748                                           0.920566        
3749                                           0.028428        
3750                                           0.028158        
3751                                           0.119596        
3752                                           0.374140        

      Ligand derived specificity of total expression value  \
0                                              0.114884      
1                                              0.059150      
2                                              0.114884      
3                                              0.144369      
4                                              0.059150      
...                                                 ...      
3748                                           0.775378      
3749                                           0.050078      
3750                                           0.001509      
3751                                           0.007585      
3752                                           0.025908      

      Receptor-expressing cells  Receptor detection rate  \
0               

# Conduct statistical tests to see diff between groups:

In [7]:

# LR_pairs = [tuple(v) for v in print_df[['Ligand symbol', 'Receptor symbol']].values.tolist()]
# unique_LR_paris = sorted(set(LR_pairs), key=LR_pairs.index)

# # print_df

# for LR_pair in unique_LR_paris[:1]:
#     print(LR_pair)

#     curr_pair_df = print_df[(print_df['Ligand symbol']==LR_pair[0]) & (print_df['Receptor symbol']==LR_pair[1])]
#     sending_clusters = list(set(curr_pair_df['Sending cluster'].tolist()))
#     target_clusters = list(set(curr_pair_df['Target cluster'].tolist()))

# target_clusters[2].split('_')    

In [8]:
EXPERIMENT_NUM = 1


if EXPERIMENT_NUM == 1:
    EXPERIMENT_NAME = r'NR_mucosal_vs_NR_cutaneous'
    FEATURE = 'Melanoma type' # 'response' | 'Melanoma type'
elif EXPERIMENT_NUM == 2:
    EXPERIMENT_NAME = r'NR_mucosal_vs_R_cutaneous'
    FEATURE = 'response'# 'response' | 'Melanoma type'
elif EXPERIMENT_NUM == 3:
    EXPERIMENT_NAME = r'NR_mucosal_vs_R_AND_NR_cutaneous'
    FEATURE = 'Melanoma type' # 'response' | 'Melanoma type'
if FEATURE == 'response':
    FEATURE_A = 'R'
    FEATURE_B = 'NR'
    FEATURE_G = 'not in use for now'
else:
    FEATURE_A = 'Cutaneous'
    FEATURE_B = 'Mucosal'
    FEATURE_G = 'other'    
    
melanoma_clinical_data = get_constant_cohort(EXPERIMENT_NUM) #get_clinical_data(71)#ICI=True, after_biopsy='ICI')
res_melanoma_clinical_data = melanoma_clinical_data.set_index('Patient id')
print(melanoma_clinical_data.iloc[:,:-1].head(5))
print(f'Num of samples in table is {len(melanoma_clinical_data)}')
print(res_melanoma_clinical_data[FEATURE].value_counts())

response_samples = melanoma_clinical_data[melanoma_clinical_data[FEATURE] == FEATURE_A]['Patient id'].tolist()
non_response_samples = melanoma_clinical_data[melanoma_clinical_data[FEATURE] == FEATURE_B]['Patient id'].tolist()
no_used_samples = melanoma_clinical_data[melanoma_clinical_data[FEATURE] == FEATURE_G]['Patient id'].tolist()

print(f'number {FEATURE_A} samples: {len(response_samples)}')
print(f'number {FEATURE_B} samples: {len(non_response_samples)}')
print(f'number no label samples: {len(no_used_samples)}')

Using clinical table in path:
 /storage/md_keren/shitay/Data/tables/clinical_tables/Melanoma_clinical_data_3.22_unprotected.xlsx

and labels:
/storage/md_keren/shitay/Data/tables/clinical_tables/clinical_labels.csv
  Patient id Melanoma type response prior to biopsy   after biopsy   ICI
0        M99       Mucosal       NR             ICI  other therapy  True
1       M106     Cutaneous       NR             ICI  other therapy  True
2       M107     Cutaneous       NR             ICI  other therapy  True
3       M111       Mucosal       NR             ICI  other therapy  True
4       M120     Cutaneous       NR             ICI  other therapy  True
Num of samples in table is 13
Cutaneous    9
Mucosal      4
Name: Melanoma type, dtype: int64
number Cutaneous samples: 9
number Mucosal samples: 4
number no label samples: 0


In [9]:
IMMUNE_CLUSTER_BARCODE_MAPPING_PATH = r'/storage/md_keren/shitay/outputs/clustering/immune/summary/subcohort_1.1.22_run_1.1.22/subcohort_immune_1.1.22_clusters_mapping.csv'
CD8_CLUSTER_BARCODE_MAPPING_PATH = r'/storage/md_keren/shitay/outputs/clustering/CD8/summary/subcohort_1.1.22_run_1.1.22/subcohort_CD8_1.1.22_clusters_mapping.csv'
MYELOID_CLUSTER_BARCODE_MAPPING_PATH = r'/storage/md_keren/shitay/outputs/clustering/myeloid/summary/subcohort_1.1.22_run_1.1.22/subcohort_myeloid_1.1.22_clusters_mapping.csv'
#######################################
immune_clusters_barcodes_mapping_df = pd.read_csv(IMMUNE_CLUSTER_BARCODE_MAPPING_PATH)
immune_clusters_barcodes_mapping_df = immune_clusters_barcodes_mapping_df[immune_clusters_barcodes_mapping_df["Sample"].isin(melanoma_clinical_data['Patient id'])]

CD8_clusters_barcodes_mapping_df = pd.read_csv(CD8_CLUSTER_BARCODE_MAPPING_PATH).drop(columns=['index', 'Unnamed: 0'])
CD8_clusters_barcodes_mapping_df = CD8_clusters_barcodes_mapping_df[CD8_clusters_barcodes_mapping_df["Sample"].isin(melanoma_clinical_data['Patient id'])]

myeloid_clusters_barcodes_mapping_df = pd.read_csv(MYELOID_CLUSTER_BARCODE_MAPPING_PATH).drop(columns=['index', 'Unnamed: 0'])
myeloid_clusters_barcodes_mapping_df = myeloid_clusters_barcodes_mapping_df[myeloid_clusters_barcodes_mapping_df["Sample"].isin(melanoma_clinical_data['Patient id'])]

In [10]:
EXEC_DIR = r'/storage/md_keren/shitay/outputs/cNMF/executions/tumor_runs/subcohort_1.1.22'
selected_K = 20
local_density_threshold = '0.10'
number_of_genes = 2000
n_replicates = 200
RUN_RANGE = '20'
N_PROG = 20

GEP_df, R_GEP_df, NR_GEP_df = get_GEP_dfs(melanoma_clinical_data,
                    EXEC_DIR,
                    RUN_RANGE,
                    selected_K,
                    number_of_genes,
                    n_replicates,
                    local_density_threshold,
                    FEATURE,
                    FEATURE_A,
                    FEATURE_B)
GEP_df = filter_contaminated_cells_out_of_GEP_DF(GEP_df, 14, contaminated_cells_path =r'/storage/md_keren/shitay/Data/tables/GEP/subcohort_1.1.22/contaminated_cells_list/GEP14_contaminated_cells.csv')
R_GEP_df = filter_contaminated_cells_out_of_GEP_DF(R_GEP_df, 14, contaminated_cells_path =r'/storage/md_keren/shitay/Data/tables/GEP/subcohort_1.1.22/contaminated_cells_list/GEP14_contaminated_cells.csv')
NR_GEP_df = filter_contaminated_cells_out_of_GEP_DF(NR_GEP_df, 14, contaminated_cells_path =r'/storage/md_keren/shitay/Data/tables/GEP/subcohort_1.1.22/contaminated_cells_list/GEP14_contaminated_cells.csv')

In [11]:
COHORT_PATH = r'/storage/md_keren/shitay/Data/droplet_seq/M97_M173/subcohort/normalized/16.3.22/subcohort_normalized_1.1.22_protein_coding_genes.pkl'
cohort = pickle.load(open(COHORT_PATH, 'rb'))

In [12]:
immune_cohort = cohort.filter_cells_by_property('is_immune', True)
tumor_cohort = cohort.filter_cells_by_property('is_cancer', True)
CD8_cohort = get_requested_subset(cohort, 'CYTOTOXIC_T_CELLS')
mye_cohort = get_requested_subset(cohort, 'MYELOIDS')


Will be performed on cytotoxic T cells
Will be performed on myeloids
IMMUNE CLUSTERING in path 	 /storage/md_keren/shitay/outputs/clustering/immune/summary/4.11.21/immune_kmeans_integrated_4.11.21_clusters_mapping.csv 	 is used for defining which cells are myeloids
Cluster 5 is defined to be myeloids


### Build cohorts by clusters and GEPs

In [13]:
GEP_indexes = list(GEP_df.columns[4:])
GEPS_dic = {g_i:{} for g_i in GEP_indexes}
for GEP_curr_index in GEP_indexes:
    R_cells_ids = R_GEP_df[R_GEP_df['associated program']==GEP_curr_index][['sample', 'barcode']]
    NR_cells_ids = NR_GEP_df[NR_GEP_df['associated program']==GEP_curr_index][['sample', 'barcode']]
    
    NR_cohort = tumor_cohort.get_subset_by_identifiers(NR_cells_ids['sample'], NR_cells_ids['barcode'])
    R_cohort = tumor_cohort.get_subset_by_identifiers(R_cells_ids['sample'], R_cells_ids['barcode'])
   
    GEPS_dic[GEP_curr_index]['R'] = R_cohort
    GEPS_dic[GEP_curr_index]['NR'] = NR_cohort

In [14]:
CD8_cluster_indexes = list(set(CD8_clusters_barcodes_mapping_df.Cluster))
CD8_dic = {c_i:{} for c_i in CD8_cluster_indexes}
R_CD8_clusters_df = CD8_clusters_barcodes_mapping_df[CD8_clusters_barcodes_mapping_df.Sample.isin(response_samples)]
NR_CD8_clusters_df = CD8_clusters_barcodes_mapping_df[CD8_clusters_barcodes_mapping_df.Sample.isin(non_response_samples)]


for cluster_curr_index in CD8_cluster_indexes:
    print(cluster_curr_index)
    R_cells_ids = R_CD8_clusters_df[R_CD8_clusters_df.Cluster == cluster_curr_index][['Sample', 'Barcode']]
    NR_cells_ids = NR_CD8_clusters_df[NR_CD8_clusters_df.Cluster == cluster_curr_index][['Sample', 'Barcode']]
    
    
    R_cohort = CD8_cohort.get_subset_by_identifiers(R_cells_ids['Sample'], R_cells_ids['Barcode'])
    NR_cohort = CD8_cohort.get_subset_by_identifiers(NR_cells_ids['Sample'], NR_cells_ids['Barcode'])

    CD8_dic[cluster_curr_index]['R'] = R_cohort
    CD8_dic[cluster_curr_index]['NR'] = NR_cohort

1
2
3
4
5


In [15]:
myeloid_cluster_indexes = list(set(myeloid_clusters_barcodes_mapping_df.Cluster))
myeloid_dic = {c_i:{} for c_i in myeloid_cluster_indexes}
R_myeloid_clusters_df = myeloid_clusters_barcodes_mapping_df[myeloid_clusters_barcodes_mapping_df.Sample.isin(response_samples)]
NR_myeloid_clusters_df = myeloid_clusters_barcodes_mapping_df[myeloid_clusters_barcodes_mapping_df.Sample.isin(non_response_samples)]


for cluster_curr_index in myeloid_cluster_indexes:
    print(cluster_curr_index)
    R_cells_ids = R_myeloid_clusters_df[R_myeloid_clusters_df.Cluster == cluster_curr_index][['Sample', 'Barcode']]
    NR_cells_ids = NR_myeloid_clusters_df[NR_myeloid_clusters_df.Cluster == cluster_curr_index][['Sample', 'Barcode']]
    
    
    R_cohort = mye_cohort.get_subset_by_identifiers(R_cells_ids['Sample'], R_cells_ids['Barcode'])
    NR_cohort = mye_cohort.get_subset_by_identifiers(NR_cells_ids['Sample'], NR_cells_ids['Barcode'])

    myeloid_dic[cluster_curr_index]['R'] = R_cohort
    myeloid_dic[cluster_curr_index]['NR'] = NR_cohort

1
2
3
4
5
6
7
8
9


#### Go through strong_signal_df

In [19]:
root_dic = {'GEP':GEPS_dic, 'Myeloid':myeloid_dic, 'CD8':CD8_dic}

In [60]:
edited_strong_signals_df = strong_signals_df.copy()

edited_strong_signals_df[f'Target cluster ({FEATURE_A}) - number of cells'] = None
edited_strong_signals_df[f'Target cluster ({FEATURE_B}) - number of cells'] = None
edited_strong_signals_df[f'Target cluster ({FEATURE_A}) - % cells expressing'] = None
edited_strong_signals_df[f'Target cluster ({FEATURE_B}) - % cells expressing'] = None
edited_strong_signals_df['Receptor - pval'] = None

edited_strong_signals_df[f'Sending cluster ({FEATURE_A}) - number of cells'] = None
edited_strong_signals_df[f'Sending cluster ({FEATURE_B}) - number of cells'] = None
edited_strong_signals_df[f'Sending cluster ({FEATURE_A}) - % cells expressing'] = None
edited_strong_signals_df[f'Sending cluster ({FEATURE_B}) - % cells expressing'] = None
edited_strong_signals_df['Ligand - pval'] = None



In [61]:
def get_statistics_of_exp(R_expression, NR_expression):
    R_n_cells = len(R_expression)
    NR_n_cells = len(NR_expression)
    R_n_cells_expressing = sum(R_expression > gene_expression_threshold)
    NR_n_cells_expressing = sum(NR_expression > gene_expression_threshold)
    R_per_expressing = R_n_cells_expressing/ R_n_cells if R_n_cells!=0 else 0
    NR_per_expressing = NR_n_cells_expressing / NR_n_cells if NR_n_cells!=0 else 0
    
    oddsratio, pvalue = stats.fisher_exact([[R_n_cells_expressing,
                                            NR_n_cells_expressing],
                                            [R_n_cells - R_n_cells_expressing,
                                                     NR_n_cells - NR_n_cells_expressing]])

    return R_n_cells, NR_n_cells, R_per_expressing, NR_per_expressing, pvalue

In [62]:
gene_expression_threshold = 1

for row_index, current_signal in edited_strong_signals_df.iterrows():


    if not current_signal['Ligand symbol'] in cohort.gene_names or not current_signal['Receptor symbol'] in cohort.gene_names:        
        continue
    ligand_gene_index = cohort.gene_names.index(current_signal['Ligand symbol'])
    receptor_gene_index = cohort.gene_names.index(current_signal['Receptor symbol'])


    cluster_name = current_signal['Sending cluster'].split('_')[0]
    if cluster_name=='GEP':
        cluster_idx = int(current_signal['Sending cluster'].split('_')[1][1:])
    else:
        cluster_idx = int(current_signal['Sending cluster'].split('_')[2])

    R_counts = root_dic[cluster_name][cluster_idx]['R'].counts
    NR_counts = root_dic[cluster_name][cluster_idx]['NR'].counts

    R_n_cells, NR_n_cells, R_per_expressing, NR_per_expressing, pval = get_statistics_of_exp(R_counts[:, ligand_gene_index], NR_counts[:, ligand_gene_index])

    edited_strong_signals_df.at[row_index, [f'Sending cluster ({FEATURE_A}) - number of cells']] =  R_n_cells
    edited_strong_signals_df.at[row_index, [f'Sending cluster ({FEATURE_B}) - number of cells']] = NR_n_cells
    edited_strong_signals_df.at[row_index, [f'Sending cluster ({FEATURE_A}) - % cells expressing']] = R_per_expressing
    edited_strong_signals_df.at[row_index, [f'Sending cluster ({FEATURE_B}) - % cells expressing']] = NR_per_expressing
    edited_strong_signals_df.at[row_index, ['Ligand - pval']] = pval


    cluster_name = current_signal['Target cluster'].split('_')[0]
    if cluster_name=='GEP':
        cluster_idx = int(current_signal['Target cluster'].split('_')[1][1:])
    else:
        cluster_idx = int(current_signal['Target cluster'].split('_')[2])

    R_counts = root_dic[cluster_name][cluster_idx]['R'].counts
    NR_counts = root_dic[cluster_name][cluster_idx]['NR'].counts

    R_n_cells, NR_n_cells, R_per_expressing, NR_per_expressing, pval = get_statistics_of_exp(R_counts[:, receptor_gene_index], NR_counts[:, receptor_gene_index])
    edited_strong_signals_df.at[row_index, [f'Target cluster ({FEATURE_A}) - number of cells']] = R_n_cells
    edited_strong_signals_df.at[row_index, [f'Target cluster ({FEATURE_B}) - number of cells']] = NR_n_cells
    edited_strong_signals_df.at[row_index, [f'Target cluster ({FEATURE_A}) - % cells expressing']] = R_per_expressing
    edited_strong_signals_df.at[row_index, [f'Target cluster ({FEATURE_B}) - % cells expressing']] = NR_per_expressing
    edited_strong_signals_df.at[row_index, ['Receptor - pval']] = pval



In [63]:
edited_strong_signals_df = edited_strong_signals_df[~((edited_strong_signals_df['Receptor - pval'].isnull()) | (edited_strong_signals_df['Ligand - pval'].isnull()))]

In [64]:
edited_strong_signals_df['Receptor - qval'] = multipletests_fdr(edited_strong_signals_df['Receptor - pval'])[1]
edited_strong_signals_df['Ligand - qval'] = multipletests_fdr(edited_strong_signals_df['Ligand - pval'])[1]


In [65]:
edited_strong_signals_df = edited_strong_signals_df[(edited_strong_signals_df['Receptor - qval'] < 0.05) | (edited_strong_signals_df['Ligand - qval']<0.05)]

In [85]:
edited_strong_signals_df = edited_strong_signals_df.iloc[:, [0, 3, 1, 2]+list(range(25,29))+list(range(20,24))+[31, 30]+list(range(4,19))+[29, 24]]

In [100]:
edited_strong_signals_df.to_excel(r'/storage/md_keren/shitay/garbage/dd3.xlsx')

In [102]:
# edited_strong_signals_df

# pairs of a specific group (NR_cutaneous/NR_mucosal/R_cutaneous):

In [36]:
groups = ['GEP_#13', 'Myeloid_cluster_4', 'Myeloid_cluster_5', 'CD8_cluster_1'] # NR_mucosal
# groups = ['GEP_#2', 'GEP_#17', 'CD8_cluster_3', 'CD8_cluster_4'] # NR_cutaneous
# groups = ['GEP_#18', 'Myeloid_cluster_2', 'Myeloid_cluster_7', 'Myeloid_cluster_9','CD8_cluster_5'] # R_cutaneous

two_sides = True
NUM_OF_ROWS = 10 # None

########################################
pd.set_option('display.max_rows', NUM_OF_ROWS)
if two_sides:
    res_df = print_df[print_df['Sending cluster'].isin(groups) & print_df['Target cluster'].isin(groups)]
else:
    res_df = print_df[print_df['Sending cluster'].isin(groups) | print_df['Target cluster'].isin(groups)]
res_df

Sending cluster Ligand symbol Receptor symbol     Target cluster  \
126             GEP_#13           MIF            CD74  Myeloid_cluster_5   
192             GEP_#13           MIF            CD74      CD8_cluster_1   
277             GEP_#13           MIF            CD74  Myeloid_cluster_4   
438   Myeloid_cluster_4           MIF            CD74  Myeloid_cluster_5   
534   Myeloid_cluster_5           MIF            CD74  Myeloid_cluster_5   
...                 ...           ...             ...                ...   
3380            GEP_#13          PSAP            LRP1  Myeloid_cluster_5   
3408  Myeloid_cluster_4           GRN        TNFRSF1B  Myeloid_cluster_5   
3432            GEP_#13         SPON2           ITGB2  Myeloid_cluster_4   
3443            GEP_#13          CALR            LRP1  Myeloid_cluster_4   
3459            GEP_#13          CALR            LRP1  Myeloid_cluster_5   

      Ligand-expressing cells  Ligand detection rate  \
126                      2946               0.990918   
192                      2946               0.990918   
277                      2946               0.990918   
438                       788               0.955152   
534                       674               0.917007   
...                       ...                    ...   
3380                     2308               0.776320   
3408                      640               0.775758   
3432                     2127               0.715439   
3443                     2264               0.761520   
3459                     2264               0.761520   

      Ligand average expression value  Ligand total expression value  \
126                          5.668784                   16853.295504   
192                          5.668784                   16853.295504   
277                          5.668784                   16853.295504   
438                          3.481804                    2872.488705   
534                          3.148925                    2314.459796   
...                               ...                            ...   
3380                         1.668917                    4961.689572   
3408                         1.897211                    1565.198715   
3432                         1.641887                    4881.329699   
3443                         1.524391                    4532.013639   
3459                         1.524391                    4532.013639   

      Ligand derived specificity of average expression value  \
126                                            0.045169        
192                                            0.045169        
277                                            0.045169        
438                                            0.027743        
534                                            0.025091        
...                                                 ...        
3380                                           0.024804        
3408                                           0.049503        
3432                                           0.155244        
3443                                           0.027601        
3459                                           0.027601        

      Ligand derived specificity of total expression value  \
126                                            0.059150      
192                                            0.059150      
277                                            0.059150      
438                                            0.010082      
534                                            0.008123      
...                                                 ...      
3380                                           0.041165      
3408                                           0.023283      
3432                                           0.174633      
3443                                           0.041345      
3459                                           0.041345      

      Receptor-expressing cells  Receptor d

# DEBUG

#### 

Sending cluster -	The cluster that expresses the ligand.

Target cluster -	The cluster that expresses the receptor.

Ligand/receptor symbol -	The official gene symbol of the detected ligand/receptor.

Ligand/receptor-expressing cells -  	Number of cells ligand/receptor is detected in.

Ligand/receptor detection rate	-   The ratio of cells that expressed the ligand/receptor to total cells in the cluster.

Ligand/receptor average expression value -	The average expression level of the ligand/receptor in the cluster.

Ligand/receptor derived specificity of average expression value	-   The ratio of the average expression level of the ligand/receptor in the cluster to the sum of the average expression levels of the ligand/receptor in every cluster.

Ligand/receptor total expression value	-  The total expression level of the ligand/receptor in the cluster.

Ligand/receptor derived specificity of total expression value - 	The ratio of the total expression level of the ligand/receptor in the cluster to the sum of the total expression levels of the ligand/receptor in every cluster.

Edge average expression weight - 	The product of average expression levels of the ligand and the receptor in the corresponding cluster(s).

Edge average expression derived specificity - 	The product of the ligand and receptor derived specificity of average expression values.

Edge total expression weight -  	The product of total expression levels of the ligand and the receptor in the corresponding cluster(s)..

Edge total expression derived specificity - 	The product of the ligand and receptor derived specificity of total expression values.


In [576]:
sum(myeloid_clusters_barcodes_mapping_df.Cluster==2)

143

In [579]:
myeloid_clusters_barcodes_mapping_df = pd.read_csv(MYELOID_CLUSTER_BARCODE_MAPPING_PATH).drop(columns=['index', 'Unnamed: 0'])
# myeloid_clusters_barcodes_mapping_df = myeloid_clusters_barcodes_mapping_df[myeloid_clusters_barcodes_mapping_df["Sample"].isin(melanoma_clinical_data['Patient id'])]
myeloid_clusters_barcodes_mapping_df
sum(myeloid_clusters_barcodes_mapping_df.Cluster==2)

1514